# Reglas de asociación y patrones secuenciales
---
Práctica de laboratorio 2 - Convocatoria Extraordinaria

**Grupo A08**

**Fecha de entrega:** 20/06/2024, 23:59

---
---

# Práctica 2: Patrones Secuenciales

El objetivo de esta práctica será sacar las conclusiones obtenidas tras la aplicación del algoritmo patrones secuenciales con distintas configuraciones.

Para la realización de la práctica serán necesarias las siguientes librerías:

In [54]:
#pip install gsppy

In [55]:
pip install gsp_python

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gsp_python
from gsp_python.gsp import GSP
#from gsppy.gsp import GSP

Se importa el archivo de datos que recoge toda la información para el desarrollo de la práctica, `Online Retail.xlsx`.

In [57]:
data = pd.read_excel('Online Retail.xlsx')
print(data.shape)
data.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


El conjunto de datos contiene productos (definidos por la columna `StockCode` y `Description`) comprados por diferentes clientes (definidos por la columna `CustomerID`) en diferentes momentos (definidos por el campo `InvoiceDate`).

La información recogida en cada una de las 8 variables es:

*   **InvoiceNo**: Es el identificador de cada transacción.

*   **StockCode**: Es el identificador de los productos.

*   **Description**: Es la descripción del producto.

*   **Quantity**: Es la cantidad de productos vendidos en un momento concreto.

*   **InvoiceDate**: Es el indicador de la fecha y hora de la transacción.

*   **UnitPrice**: Es el precio unitario del producto.

*   **CustomerID**: Es el identificador del cliente.

*   **Country**: Es eel nombre del país donde se realizó la transacción.

## Preprocesamiento de los datos

En primer lugar se hará el preprocesamiento de los pasos. En esta sección se limpiarán los datos de manera que se desecharán los datos duplicados, se estudiarán datos incompletos y la existencia de outliers.

Con el objetivo de que cada transacción se cuente solo una vez y evitar que sobreestime la frecuencia de ciertos patrones, se eliminan elementos repetidos.

In [58]:
data = data.drop_duplicates()
print(data.shape)
data.head()

(536641, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


A continuación se mostrarán aquellos datos incompletos:

In [59]:
data.isna().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

Se observa que tan solo hay elementos vacíos en la columna `Description`, sin embargo, al tratarse de una columna que solo aporta una pequeña descripción del producto comprado, como no se va a trabajar con ella y, por tanto, no va a afectar al ejercicio, no es necesario la eliminación de aquellas filas o su transformación.

Debido a que no todas las variables son necesarias para el análisis que se va a realizar, se eliminan aquellas que no presentar información adicional.

In [60]:
data = data.drop(columns = ['Description', 'UnitPrice'])
print(data.shape)
data.head()

(536641, 6)


,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,17850,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,17850,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,17850,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,17850,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,17850,United Kingdom


Como no hay elementos vacíos en las columnas principales se puede continuar con el estudio de los posibles outliers que presente el conjunto de datos.

Los outliers son aquellos valores menores o iguales a 0, ya que no aportan información real al análisis, y aquellos valores por encima del valor máximo calculado.

In [61]:
# Outliers por debajo del 0
data = data[data['Quantity'] > 0]

# Outliers por encima del máximo
q1 = data['Quantity'].describe()[4]
q3 = data['Quantity'].describe()[6]

data = data[data['Quantity'] < q3 + 1.5 * (q3 - q1)]
print(data.shape)
data.head()

(498595, 6)


,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID,Country
0,536365,85123A,6,2010-12-01 08:26:00,17850,United Kingdom
1,536365,71053,6,2010-12-01 08:26:00,17850,United Kingdom
2,536365,84406B,8,2010-12-01 08:26:00,17850,United Kingdom
3,536365,84029G,6,2010-12-01 08:26:00,17850,United Kingdom
4,536365,84029E,6,2010-12-01 08:26:00,17850,United Kingdom


## Transformación de los datos

En esta sección se aplicarán diferentes operaciones con el objetivo de adaptar la base de datos al algoritmo que se quiere utilizar, este caso, el GSP.

Aparentemente, parece que la variable InvoiceDate está en formato fecha, pero aun así se aplicará la función `to_datetime` para asegurar que esté en el formato correspondiente.

In [62]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['Date'] = data['InvoiceDate'].dt.date

Al haber hecho algunas pruebas previamente, se ha observado que el algoritmo no funciona correctamente ya que en vez de estudiar los patrones de cada artículo, encuentra patrones en cada dígito de cada código de Stock, ante esto se ha propuesto clasificar cada código de Stock como un único valor, del 0 al 3925.

In [63]:
# Crear un mapeo de StockCode a números únicos
stockcode_to_number = {stockcode: i for i, stockcode in enumerate(data['StockCode'].unique())}

# Aplicar el mapeo para asignar números únicos a StockCode
data['StockCode_Encoded'] = data['StockCode'].map(stockcode_to_number)

Además, el hecho de que encuentre patrones por cada dígito del código (3, 9, 2, 5) en vez del código completo (3925) se trabajará únicamente con los 10 primeros productos:

In [64]:
data['StockCode_Encoded'] = pd.to_numeric(data['StockCode_Encoded'], errors='coerce')
selected_rows = data.query('StockCode_Encoded >= 0 and StockCode_Encoded <= 9')

Finalmente, se crearán la lista de secuencias para cada cliente, para ello se definen las transacciones separadas. Se empleará la operación `groupby` que agrupará el StockCode en función del ID de cada cliente y la fecha que indica cuándo compró el producto. Además, el objetivo del algoritmo GSP es extraer patrones frecuentes relacionados con el tiempo, es por esto que es necesario ordenar el dataset en función del cliente y la fecha mediante la función `sort_values`:

In [65]:
selected_rows = selected_rows.sort_values(by=['CustomerID', 'InvoiceDate'])
customer_transactions = selected_rows.groupby('CustomerID').apply(lambda x: x[['Date', 'StockCode_Encoded']].values.tolist()).tolist()

In [66]:
# Convertir cada transacción en un conjunto de ítems
sequences = []
for customer in customer_transactions:
    sequence = []
    for transaction in customer:
        # Cada transacción es una lista con [InvoiceDate, StockCode]
        # Nos interesa solo StockCode
        items = str(transaction[1])
        sequence.append(items)
    sequences.append(sequence)

In [67]:
len(sequences)

1525

Se observa cómo quedan las primeras 5 secuencias:

In [68]:
print(sequences[:5])

[['3', '4'], ['5'], ['9'], ['9', '8', '3'], ['9']]


Una vez que se tienen los datos transformados, se aplicará el algoritmo GSP para analizar los resultados obtenidos y sacar distintas conclusiones.

## Aplicación del algoritmo GSP

Se aplica el algoritmo GSP con distintos valores de soporte para estudiar las diferencias en los patrones generados.

### Valor soporte de 0.05

En el primer caso se verá qué patrones se encuentran con un soporte mínimo de `0.05`.

In [69]:
gsp_instance = GSP(sequences, minsup=0.05)
patterns = gsp_instance.run_gsp()

for pattern in patterns:
    print(pattern)

([['3']], 210)
([['4']], 223)
([['5']], 188)
([['9']], 213)
([['8']], 247)
([['0']], 772)
([['1']], 148)
([['6']], 80)
([['2']], 126)
([['7']], 288)
([['8'], ['7']], 89)
([['0'], ['0']], 349)
([['0'], ['7']], 90)
([['7'], ['8']], 87)
([['7'], ['7']], 81)
([['0'], ['0'], ['0']], 191)
([['0'], ['0'], ['0'], ['0']], 115)


Empleando este soporte mínimo se observa que se han encontrado patrones hasta `k = 4`.


*   Para `k = 1` se observa que el producto más comprado es el producto `0` pues se encuentra en 772 secuencias de 1525 que hay en total, es decir que en el 50,6% de los casos se compra este producto. Sin embargo, el producto `6` es el que menos aparece, comprándose en un 5,25% de los casos.
*   Para `k = 2` se obtiene que la combinación más usada es del producto `0` consigo mismo, siendo así en el 22,88% de los casos. Además de ver otras combinaciones como el producto `8` con el `7` o el `0` con el `7`.
*   Finalmente, para `k = 3` y `k = 4` se observa que el producto `0` suele comprarse.

Con esto se llega a la conclusión de que tras comprar el producto `8` se compra el producto `7` o después de comprar el producto `0` se compra el producto `7`, por tanto se puede aconsejar la compra del producto `7` cuando se compra el `8` y viceversa, al igual que se puede aconsejar la compra del producto `7` si se compra el `0`.





### Valor soporte de 0.1

En el segundo caso se verá qué patrones se encuentran con un soporte mínimo de `0.1`.

In [70]:
gsp_instance1 = GSP(sequences, minsup=0.1, mingap=1, maxgap=2, maxspan=5)
patterns1 = gsp_instance1.run_gsp()

for pattern1 in patterns1:
    print(pattern1)

([['3']], 210)
([['4']], 223)
([['5']], 188)
([['9']], 213)
([['8']], 247)
([['0']], 772)
([['7']], 288)
([['0'], ['0']], 196)


Con un soporte mínimo mayor se observa que se han encontrado patrones hasta `k = 2`.

A diferencia del anterior caso, en este desaparecen, para `k = 1`, los productos `1`, `2` y `6` y para `k = 2` la única combinación existente es la del producto `0` consigo mismo, por lo que no nos aporta demasiada información.

### Valor soporte de 0.3

En el tercer y último caso se verá qué patrones se encuentran con un soporte mínimo de `0.01`.

In [71]:
gsp_instance2 = GSP(sequences, minsup=0.01, mingap=1, maxgap=2, maxspan=5)
patterns2 = gsp_instance2.run_gsp()

for pattern2 in patterns2:
    print(pattern2)

([['3']], 210)
([['4']], 223)
([['5']], 188)
([['9']], 213)
([['8']], 247)
([['0']], 772)
([['1']], 148)
([['6']], 80)
([['2']], 126)
([['7']], 288)
([['3'], ['3']], 21)
([['3'], ['0']], 28)
([['3'], ['7']], 21)
([['4'], ['3']], 17)
([['4'], ['4']], 17)
([['4'], ['0']], 31)
([['5'], ['5']], 25)
([['5'], ['0']], 23)
([['9'], ['9']], 29)
([['9'], ['0']], 24)
([['8'], ['4']], 17)
([['8'], ['8']], 25)
([['8'], ['0']], 22)
([['8'], ['7']], 26)
([['0'], ['3']], 31)
([['0'], ['4']], 30)
([['0'], ['5']], 17)
([['0'], ['9']], 29)
([['0'], ['8']], 45)
([['0'], ['0']], 196)
([['0'], ['1']], 26)
([['0'], ['2']], 28)
([['0'], ['7']], 53)
([['1'], ['0']], 24)
([['6'], ['0']], 17)
([['2'], ['0']], 37)
([['2'], ['2']], 24)
([['7'], ['3']], 23)
([['7'], ['8']], 27)
([['7'], ['0']], 33)
([['7'], ['7']], 36)
([['0'], ['0'], ['8']], 17)
([['0'], ['0'], ['0']], 65)
([['0'], ['0'], ['7']], 16)
([['0'], ['0'], ['0'], ['0']], 26)


Con un soporte mínimo menor que el inicial se observa que también se llega hasta `k = 4`. Además, se obtienen más combinaciones para `k = 2` y `k = 3`.

Para `k = 2` se observa que en la mayoría de casos lo normal tras comprar un producto es que también se compre el producto `0` y por tanto sería adecuado su recomendación tras la compra de cualquiera.

Por otro lado, para `k = 3` se recomienda que tras la segunda compra del producto `0`, se recomienden el producto `7` u `8`.

## Conclusiones

Finalmente se observa que a medida que el soporte mínimo aumenta, el número de patrones disminuye y con ello el número de combinaciones de los productos.

Además para este caso se observa que el producto `0` es el más comprado siendo mayormente combinado con el `8` y `7`.

Sin embargo, la base de datos, debido a complicaciones de computación ha sido modificada y adaptada para el correcto funcionamiento del algoritmo, haciendo que los resultados no sean del todo precisos, sin embargo, como ya ha sido explicado, se ha tenido que recurrir a esta solución ya que no cogía como debía los códigos de Stock.